In [1]:
import pandas as pd
import numpy as np
from numpy import random

In [2]:
# Create the list of file names: 
filenames=['PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.28_R0_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.64_R0_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.28_R1_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.64_R1_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.28_R2_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.64_R2_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.28_R3_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.64_R3_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.28_R4_soln.txt',
           'PS2000_BF10_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.64_R4_soln.txt']
dataframes = []
for filename in filenames:
    dataframes.append(pd.read_table(filename,header=None))

In [ ]:
# Create the list of file names: 
filenames=['PS1000_BF20_Prob2_q0.5_a1_0.95_b1_1.25_a2_1.15_b2_0.90_cl_1.28_R1_soln.txt']
dataframes = []
for filename in filenames:
    dataframes.append(pd.read_table(filename,header=None))

In [ ]:
#dataframes[0] #first data frame
#data
#0 project index,  1 xi (selection or not), 2 pi(cancelllation prob), 3 bi (project budget)

In [ ]:
#data[3].sum() #total sum of all projects

In [3]:
bf=0.1  #10 means 0.1, 20 means 0.2
q=0.5 #underutilization prob,

alfa1=0.95 #do not forget rau 1 and rau 2 truncation bounds
beta1=1.25
alfa2=1.15
beta2=0.9

rau1=0.4
rau2=0.7

#truncated sampling
random.seed(1)
truncated_beta1=[] #define an empty list for truncated beta for cancelletions
for i in range(0,1000001): #1,000,000 sampling
    r=round(np.random.beta(alfa1,beta1),4)
    if r<=rau1: truncated_beta1.append(r)
#print(truncated_beta1)


truncated_beta2=[] #define an empty list for truncated beta for underspending
for i in range(0,1000001):
    r=round(np.random.beta(alfa2,beta2),4)
    if r>=rau2: truncated_beta2.append(r)
#print(truncated_beta2)

In [4]:
#every solution file (i.e.data) is assigned a dataframe and then read and simulated from dataframes
for data in dataframes:
    #project status assigment
    #random.random generates uniformly distributed numbers btw 0,1 
    #random.uniform generates uniformly distributed numbers btw low and high
    num_of_replications=10000 #this is for simulation size
    indicator=0 
    confidence_level=0
    utilization=0
    avg_utilization=0

    np.random.seed(1)
    
    for i in range(0,num_of_replications):
        
        #assign status in new col 4: 0 means cancelation, -1 means underutilization, 1 means exact spending
        data[4]=data[2].map(lambda x: 0 if x >= np.random.random() else (-1 if q+x>=np.random.random() else 1))

        #assign spending rate according to the status in new col5
        #cancellation (i.e. 0) beta (0,tau1), underutilization (i.e. -1) beta (tau2,1), o/w 1 
        data[5]=data[4].map(lambda x: truncated_beta1[random.randint(0,10000)] if x==0 else (truncated_beta2[random.randint(0,10000)] if x==-1 else 1))
    
        #compute total simulated spending by xi*bi*spending ratio (i.e. columns 1*3*5)
        data[6]=data[1]*data[3]*data[5]

        # check total used budget is less than or equal to available budget
        #available budget is bf percent of total budget
        #print(data[6].sum()) 
        #print(data[3].sum()*bf)
        if data[6].sum()<=data[3].sum()*bf: indicator+=1

        #compute budget utilization 
        utilization+=data[6].sum()/(data[3].sum()*bf)

    #compute avg utilization and condifence level
    avg_utilization=utilization/num_of_replications
    confidence_level=indicator/num_of_replications
    #print("confidence level is %0.4f and avg utilization is %0.4f" % (confidence_level,avg_utilization))
    print("%0.4f    %0.4f" % (confidence_level,avg_utilization))

0.9760    0.9698
0.9921    0.9639
0.9764    0.9691
0.9918    0.9635
0.9796    0.9701
0.9941    0.9646
0.9778    0.9698
0.9918    0.9641
0.9807    0.9702
0.9936    0.9646


In [ ]:
#data.drop([7],axis=1,inplace=True)
data

In [ ]:
data[3].sum()*bf#total sum of all projects

In [ ]:
data[6].sum()